In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import dask.array as da


In [2]:
f = h5py.File(r"X:\Angio\2022-11-22_145205\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-right_etc.lux.h5", "r+")
print(list(f.keys()))
d = f['Data']
print(d.shape)

data = da.from_array(d, chunks =(64,64,64))

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'X:\Angio\2022-11-22_145205\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-right_etc.lux.h5', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

In [ ]:
subv = data[6:128,0:512,0:512] # do not modify!!

In [ ]:
with h5py.File("ch-1_subv.h5", "w") as f:
    dset = f.create_dataset("subv", subv.shape, dtype='uint16')
    dset.write_direct(subv)

In [5]:
l = h5py.File(r"X:\Angio\2022-11-22_145205\ch-1_subv_labkit_label.h5", "r+")
print(list(l.keys()))
label = l['t00000']['s00']['0']['cells']



['__DATA_TYPES__', 's00', 't00000']


In [ ]:
import napari

viewer = napari.view_image(subv, colormap='magma')

In [ ]:
labels_layer = viewer.add_labels(label, name='segmentation')

In [ ]:
label_closed = ndi.binary_closing(label,structure=np.ones((2,2,2)))

In [ ]:
labels_layer2 = viewer.add_labels(label_closed, name='segmentation_cl')

In [6]:
label_open = ndi.binary_opening(label,structure=np.ones((2,2,2)))
label_open = da.from_array(label_open, chunks=(64,64,64))

In [ ]:
labels_layer3 = viewer.add_labels(label_open, name='segmentation_op')

In [ ]:
labels_layer4 = viewer.add_labels(label - label_open, name='segmentation_diff')

In [ ]:
label_open.shape

In [ ]:
viewer.add_image(subv)

In [ ]:
import napari

viewer = napari.view_labels(label_open)
viewer.add_image(subv)

In [ ]:
label_open.to_hdf5('ch-1_subv_labkit_label_opened.h5', datapath = 'label/1')



In [9]:
label_open.rechunk()
label_open.to_zarr('ch-1_subv_labkit_label_opened.zarr')




In [ ]:
tifffile.imwrite('ch-1_subv_labkit_label_opened.zarr')

In [ ]:
f = h5py.File('ch-1_subv_labkit_label_opened.h5')
print(list(f.keys()))
d = f['label/1']
label_open = da.from_array(d, chunks =(64,64,64))

In [ ]:
f = h5py.File('ch-1_subv.h5')
print(list(f.keys()))
d = f['subv']
subv = da.from_array(d, chunks =(64,64,64))

In [ ]:
label_open, n_feats = ndi.label(label_open)

In [ ]:
from skimage import measure


regions = measure.regionprops(label_open)

# Extract areas of labeled regions
areas = [region.area for region in regions]

plt.hist(areas, bins =np.logspace(np.log10(0.1),np.log10(10000), 100))
plt.xlim((3,10000))
plt.xscale('log')

In [ ]:
areas

In [ ]:
label_open2 = ndi.binary_opening(label,structure=np.ones((3,3,3)))
label_open2 = da.from_array(label_open2, chunks=(64,64,64))


In [ ]:
label_open2, n_feats = ndi.label(label_open2)
regions = measure.regionprops(label_open2)

# Extract areas of labeled regions
areas = [region.area for region in regions]

plt.hist(areas, bins =np.logspace(np.log10(0.1),np.log10(10000), 100))
plt.xlim((3,10000))
plt.xscale('log')

In [ ]:
label2, n_feats = ndi.label(label)
regions = measure.regionprops(label2)

# Extract areas of labeled regions
areas = [region.area for region in regions]

plt.hist(areas, bins =np.logspace(np.log10(0.1),np.log10(10000), 100))
plt.xlim((1,10000))
plt.xscale('log')

In [ ]:
min(areas)